In [1]:
import pandas as pd
import numpy as np
from collections import Counter as ctr

In [2]:
data = pd.read_csv('spam.csv', skiprows=1, names=['type','text', 'c3', 'c4', 'c5'], delimiter=',', encoding='ISO-8859-1')

FileNotFoundError: [Errno 2] No such file or directory: 'spam.csv'

In [19]:
data['clean'] = data.text.apply(lambda x: x.lower().split())

In [20]:
data.drop(['c3', 'c4', 'c5'], axis=1, inplace=True)

In [21]:
data

,type,text,clean
0,ham,"Go until jurong point, crazy.. Available only ...","[go, until, jurong, point,, crazy.., available..."
1,ham,Ok lar... Joking wif u oni...,"[ok, lar..., joking, wif, u, oni...]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f..."
3,ham,U dun say so early hor... U c already then say...,"[u, dun, say, so, early, hor..., u, c, already..."
4,ham,"Nah I don't think he goes to usf, he lives aro...","[nah, i, don't, think, he, goes, to, usf,, he,..."
...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,"[this, is, the, 2nd, time, we, have, tried, 2,..."
5568,ham,Will Ì_ b going to esplanade fr home?,"[will, ì_, b, going, to, esplanade, fr, home?]"
5569,ham,"Pity, * was in mood for that. So...any other s...","[pity,, *, was, in, mood, for, that., so...any..."
5570,ham,The guy did some bitching but I acted like i'd...,"[the, guy, did, some, bitching, but, i, acted,..."


In [22]:
data.groupby('type').count()

,text,clean
type,,
ham,4825,4825
spam,747,747


In [23]:
type_ctr = ctr(data.type)

type_ctr['ham']/len(data), type_ctr['spam']/len(data)

(0.8659368269921034, 0.13406317300789664)

In [24]:
test = data.sample(frac=0.1)
train = data[~data.index.isin(test.index)]

train.shape, test.shape

((5015, 3), (557, 3))

### Prior probability

In [27]:
ham_spam_ctr = ctr(train.type)

def Pa(X=''):
    return ham_spam_ctr[X] / len(train)

In [68]:
Pa('ham')

0.8624127617148555

### Normalization

In [54]:
words_ctr = ctr([word for row in train.clean for word in row])

def Pb(W=''):
    if W not in words_ctr: return 0.0001
    return words_ctr[W] / sum(words_ctr.values())

### Maximum Likelihood

In [69]:
words_ham_spam = {}

for col_name in list(set(data.type)):
    sub_df = train[train.type == col_name]
    words_ham_spam[col_name] = []
    for row in sub_df.clean:
        for word in row:
            words_ham_spam[col_name].append(word)
    words_ham_spam[col_name] = ctr(words_ham_spam[col_name])

def Pba(W='', X=''):
    t = words_ham_spam[X]
    if W not in t: return 0.0000001
    return t[W] / sum(t.values())

In [70]:
Pba(W='the', X='ham')

0.016224714730290458

In [71]:
def Pab(X='', W=''):
    return Pba(W, X) * Pa(X) / Pb(W)

In [72]:
Pab(X='ham', W='the')

0.926898823413767

In [73]:
def Ps(T, X=''):
    return np.prod([Pab(X=X, W=word) for word in T])

In [74]:
Ps(['the', 'day', 'ahoythere'], 'spam')

7.03707527952389e-07

### Evaluate

In [75]:
test['ham'] = test.clean.apply(lambda x: Ps(x, 'ham'))
test['spam'] = test.clean.apply(lambda x: Ps(x, 'spam'))

In [87]:
test['winner'] = test.ham /  test.spam
test['winner'] = test.winner.apply(lambda x: 'ham' if x>1 else 'spam')

In [88]:
result = test.winner == test.type

In [89]:
sum(result) / len(test)

0.9569120287253142